In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from jupyter_chat import *
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]

In [ ]:
class MyChatbot:

    def __init__(self):
        self.systemPrompt = "You are a helpful assistant"
        self.model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7).bind_tools(tools)
        self.reset()

    def prompt(self):
        return [SystemMessage(content=self.systemPrompt)] + self.history

    def question(self, question):
        self.history.append(HumanMessage(content=question))
    
    def answer(self, question):
        ai_message = self.model.invoke(self.prompt())
        self.history.append(ai_message)
        for tool_call in ai_message.tool_calls:
            selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
            tool_output = selected_tool.invoke(tool_call["args"])
            self.history.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
            ai_message = self.model.invoke(self.prompt())
            self.history.append(ai_message)
        
    def reset(self):
        self.history = []

In [ ]:
chatbot = MyChatbot()

start_chat(chatbot)